In [1]:
# ---------------------------------------------GENERAL-----------------------------------------
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
import json
import csv
import requests
import numpy as np
import pandas as pd
import dateutil.parser
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from dateutil.tz import tz
import os
import pickle
import io
import uuid

In [2]:
def download_KASTOM_data(node):
    
    # download KASTOM data
    years = [2022]
    months = [4, 5, 6, 7, 8]
    df_list = []
    for year in years:
        for month in months:
            if month == 12:
                #print(year, month, 1)
                url1 = f'http://kastom.dotsoft.gr/data?start_date={year}/{month}/1&end_date={year + 1}/{1}/1&dev_id={node}'
                df_data1 = pd.DataFrame(requests.get(url1).json())
                df_list.append(df_data1)
            else:
                #print(year, month, month + 1)
                url1 = f'http://kastom.dotsoft.gr/data?start_date={year}/{month}/1&end_date={year}/{month + 1}/1&dev_id={node}'
                df_data1 = pd.DataFrame(requests.get(url1).json())
                df_list.append(df_data1)
    
    df = pd.concat(df_list)
    df.index = [i for i in range(len(df))]
    return df
    
def convert_GAS_sensors(df, nkastom):
    
    # depended on the sensors, provided by westsea
    metadata = pd.DataFrame()
    metadata['Node_Id'] = [i for i in range(1,26)]
    metadata['WE_NO2_elc0'] = [222,230,220,229,224,215,234,216,225,221,221,233,225,237,221,225,221,228,228,232,233,221,226,227,221]
    metadata['WE_NO2_O'] = [203,218,212,215,202,196,221,188,214,191,208,224,204,218,204,215,206,210,206,218,228,216,210,222,207]
    metadata['AE_NO2_elc0'] = [222,225,229,228,227,231,220,222,227,225,232,220,228,227,226,229,225,231,221,230,222,225,212,228,235]
    metadata['AE_NO2_O'] = [216,221,226,220,214,225,214,208,224,208,226,218,222,223,217,224,217,225,209,221,223,223,205,228,229]
    metadata['NO2_sens'] = [285,264,258,254,256,285,277,246,245,245,272,274,270,276,264,250,284,292,250,265,281,270,256,266,266]
    metadata['NO2_sens'] = metadata['NO2_sens'] /1000
    metadata['WE_OX_elc0'] = [227,234,222,233,231,230,221,228,225,220,232,238,221,218,233,217,228,224,234,222,222,226,231,216,221]
    metadata['AE_OX_elc0'] = [240,238,229,222,321,221,238,223,227,231,220, 223, 229,224,225,222,234,232,232,226,230,231,222,217,231]
    metadata['NO2_sens_OX'] = [416, 435,444,459,341,408,445,454,408,418,405,501,468,449,394,480,434,455,377,457,429,422,450,395,474]
    metadata['OX_sens'] = [300,326,327,350,381,325,326,308,304,343,325,357,334,317,318,334,334,319,276,333,347,343,324,315,328]
    metadata['OX_sens'] = metadata['OX_sens'] /1000
    metadata['NO2_sens_OX'] = metadata['NO2_sens_OX'] /1000
    
    def mv2ppb(idNo,T,WE_NO2,AE_NO2,WE_OX,AE_OX):
        this_data = metadata.iloc[idNo-1,:]
        nt1 = []
        kt = []

        for i in range(len(T)):
            if T[i] <= 30:
                kt.append(1)
            elif T[i]> 30 and T[i]<=40:
                kt.append(0.4)
            elif T[i]> 40 and T[i]<=50:
                kt.append(-0.1)
            else:
                kt.append(-4)
        for i in range(len(T)):
            if T[i] <= -20:
                nt1.append(0.9)
            elif T[i]> -10 and T[i]<=0:
                nt1.append(1.0)
            elif T[i]> 0 and T[i]<=10:
                nt1.append(1.3)
            elif T[i]> 10 and T[i]<=20:
                nt1.append(1.5)
            elif T[i]> 20 and T[i]<=30:
                nt1.append(1.7)
            elif T[i]> 30 and T[i]<=40:
                nt1.append(2)
            elif T[i]> 40 and T[i]<=50:
                nt1.append(2.5)
            else:
                nt1.append(3.7)  
        no2 = (WE_NO2 - this_data['WE_NO2_elc0'] - (this_data['WE_NO2_O'] - this_data['WE_NO2_elc0'] - this_data['AE_NO2_O'] + this_data['AE_NO2_elc0']) -  kt * (AE_NO2 - this_data['AE_NO2_elc0'])) / this_data['NO2_sens']
        no2_mv = no2*this_data['NO2_sens_OX']
        no2o3_mv = WE_OX - this_data['WE_OX_elc0'] - nt1*(AE_OX - this_data['AE_OX_elc0'])
        o3_mv = no2o3_mv - no2_mv
        o3 = o3_mv/this_data['OX_sens']

        return no2, o3, no2 + o3
    
    def O3_to_ug(ppb,T,P):
        ug = (ppb * 48 * 273 * P) / (22.4136 * (273 +T) * 1013)
        return ug

    def CO_to_ug(ppb,T,P):
        ug = (ppb * 28.01 * 273 * P) / (22.4136 * (273 +T) * 1013)
        return ug

    def NO2_to_ug(ppb,T,P):
        ug = (ppb * 46.005 * 273 * P) / (22.4136 * (273 +T) * 1013)
        return ug
    
    if nkastom <= 25:
        # keep the columns of interest
        use_data = df[['time', 'no2_aemv', 'no2_wemv', 'ox_aemv', 'ox_wemv', 
                   'temperature', 'humidity', 'pressure',
                   'pm_10','pm_2_5','pm_1_0', 'co']]
        # convert mV to ppb
        no2, o3, no2pluso3 = mv2ppb(nkastom, use_data['temperature'],use_data['no2_wemv'],use_data['no2_aemv'],use_data['ox_wemv'],use_data['ox_aemv'])

        # convert ppb to ug/m3
        o3 = O3_to_ug(o3, use_data['temperature'], use_data['pressure'])
        no2 = NO2_to_ug(no2, use_data['temperature'], use_data['pressure'])
        co = CO_to_ug(use_data['co'], use_data['temperature'], use_data['pressure'])

        use_data['o3_ug/m3'] = o3
        use_data['no2_ug/m3'] = no2
        use_data['no2pluso3_ug/m3'] = no2 + o3
        use_data['co'] = co
    else:
        # keep the columns of interest
        use_data = df[['time', 'temperature', 'humidity', 'pressure', 'pm_10','pm_2_5','pm_1_0']]
    
    # use_data.to_csv(f'~/venv/KASTOM_data/{node}_v4.csv')
    return use_data

def avg_data(df_data):
    
    df_data = df_data.replace(-1, np.nan)

    # reverse time from string to datetime object
    times = [datetime.strptime(time[:-5], '%Y-%m-%dT%H:%M:%S') for time in df_data['time']]

    # define dataframe index equal to the timestamp
    df_data.index = pd.DatetimeIndex(times)
    df_data = df_data.sort_index()

    types = df_data.dtypes
    to_drop = []
    for i, ty in enumerate(types):
        if ty != np.float64 and ty != np.int64:
            to_drop.append(types.index[i])
    new_df = df_data.drop(to_drop, axis=1)

    averaged = {}
    for col in new_df.columns:

        hours = list(new_df.index.hour)
        temp = new_df[col][0]

        new_series = []
        new_times = []
        to_be = [new_df[col][0]]

        for i in range(1, len(df_data)):
            
            if hours[i] == hours[i-1]:
                to_be.append(new_df[col][i])
            else:
                aggregation = pd.Series(to_be)#.sample(frac=0.1, replace=True, random_state=1)
                # aggregation = aggregation[(aggregation < aggregation.quantile(.95)) & (aggregation > aggregation.quantile(.05))]
                new_series.append(aggregation.median())
                new_times.append(datetime(df_data.index[i].year, # i - 1
                                          df_data.index[i].month, 
                                          df_data.index[i].day, 
                                          hours[i], 0, 0))

                to_be = []


        T = pd.Series(new_series)
        T.index = pd.DatetimeIndex(new_times)
        T = T.sort_index()

        averaged[col] = T

    return pd.DataFrame(averaged)

'''
nkastom18 is the id for the MENG AUTH TARATSA node.
my suggestion is to just run this script whenever you need to download new data. 
This script will return the data from the start of 2022 untill the date that data are available. (runs in < 5 minutes)
data3 is the dataframe you want.

'''
data = download_KASTOM_data('nkastom18')
data2 = convert_GAS_sensors(data, 18)
data3 = avg_data(data2)
data3

,no2_aemv,no2_wemv,ox_aemv,ox_wemv,temperature,humidity,pressure,pm_10,pm_2_5,pm_1_0,co,o3_ug/m3,no2_ug/m3,no2pluso3_ug/m3
2022-04-01 01:00:00,228.0,230.0,231.0,239.0,17.168890,65.734700,999.772460,49.0,29.0,11.0,301.565112,-71.945721,117.432776,45.474904
2022-04-01 02:00:00,228.0,231.0,232.0,239.0,16.526110,67.007164,999.239000,49.0,28.0,11.0,270.918812,-81.476070,117.687973,36.234918
2022-04-01 03:00:00,228.0,230.0,231.5,237.5,17.232638,61.718965,999.128950,47.5,27.0,10.5,313.376060,-76.218581,110.863347,34.625324
2022-04-01 04:00:00,229.0,231.0,232.0,239.0,17.128054,61.539986,998.284500,56.0,33.0,13.0,340.543740,-71.524223,110.806190,39.281968
2022-04-01 05:00:00,228.0,236.0,232.0,239.5,16.934165,63.182483,997.774350,64.0,36.5,15.0,674.088662,-124.025943,149.969917,18.729018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01 19:00:00,228.0,235.5,232.5,246.5,27.697919,77.214372,1006.192750,222.5,253.0,283.5,560.491116,-79.096129,142.680877,62.989423
2022-09-01 20:00:00,228.0,233.0,232.0,242.0,26.950280,80.081924,1006.430100,253.0,273.0,293.0,459.382323,-79.912394,127.197027,47.208002
2022-09-01 21:00:00,229.0,232.5,233.0,241.5,27.135559,80.173140,1006.491775,190.5,232.0,273.0,412.034462,-84.185919,120.710651,35.397353
2022-09-01 22:00:00,229.0,230.0,233.0,238.0,27.808615,75.726944,1006.699950,193.0,234.0,274.0,423.865552,-79.832186,101.465045,21.859623


In [3]:
data3.to_csv("kastom_raw.csv")